In [38]:
input = """
Method	Hardware Support	Quantization Type	WM (GB)	RM (GB)	Tokens/sec	Perplexity
Llama.cpp	NVIDIA GPU	GGUF K-Quant 2bit	2.36	3.69	102.15	6.96
-	AMD GPU	GGUF 4bit (check)	3.56	4.88	128.97	5.96
-	Apple Silicon	GGUF AWQ 4bit	3.56	4.88	129.25	5.91
-	CPU	GGUF K-Quant 4bit	3.59	4.90	109.72	5.87
-	-	GGUF 8bit	6.67	7.78	93.39	5.79
-	-	GGUF FP16	12.55	13.22	66.81	5.79
ExLlama	NVIDIA GPU	GPTQ 4bit	3.63	5.35	77.10	6.08
-	AMD GPU	-	-	-	-	-
ExLlamav2	NVIDIA GPU	EXL2 2bit	2.01	5.21	153.75	20.21
-	AMD GPU	EXL2 4bit	3.36	6.61	131.68	6.12
-	-	GPTQ 4bit	3.63	6.93	151.30	6.03
-	-	EXL2 8bit	6.37	9.47	115.81	5.76
-	-	FP16	12.55	15.09	67.70	5.73
vLLM	NVIDIA GPU	AWQ GEMM 4bit	3.62	34.55	114.43	6.02
-	AMD GPU	GPTQ 4bit	3.63	36.51	172.88	6.08
-	-	FP16	12.55	35.92	79.74	5.85
TensorRT-LLM	NVIDIA GPU	AWQ GEMM 4bit	3.42	5.69	194.86	6.02
-	-	GPTQ 4bit	3.60	5.88	202.16	6.08
-	-	INT8	6.53	8.55	143.57	5.89
-	-	FP16	12.55	14.61	83.43	5.85
TGI	AMD GPU	AWQ GEMM 4bit	3.62	7.97	30.80	6.02
-	NVIDIA GPU	AWQ GEMV 4bit	3.62	7.96	34.22	6.02
-	Intel GPU	GPTQ 4bit	3.69	39.39	34.86	6.08
-	AWS Inferentia2	FP4	12.55	17.02	34.38	6.15
-	-	NF4	12.55	17.02	33.93	6.02
-	-	INT8	12.55	11.66	5.39	5.89
-	-	FP16	12.55	17.02	34.23	5.85
MLC-LLM	NVIDIA GPU	OmniQuant 3bit	3.2	5.1	83.4	6.65
-	AMD GPU,	OmniQuant 4bit	3.8	5.7	134.2	5.97
-	CPU, WebGPU,	AWQ GEMM 4bit	3.62	6.50	23.62	6.02
-	Apple Silicon,	Q4F16	3.53	6.50	189.07	-
-	Intel GPU,	Q3F16	2.84	5.98	185.47	-
-	WASM, Adreno Mali	FP16	12.55	15.38	87.37	5.85
"""

lines = input.strip().split('\n')
data = [line.split('\t') for line in lines]


In [39]:
header = data[0]
body = data[1:]

method_name = None
hardware_support_dict = {}
possible_hardware = []

for row in body:
    if row[0] != '-':
        method_name = row[0]
    else:
        row[0] = method_name

    if row[1] != '-':
        if method_name not in hardware_support_dict:
            hardware_support_dict[method_name] = []
        for item in row[1].split(','):
            if item != '':
                hardware_support_dict[method_name].append(item.strip())

for values in hardware_support_dict.values():
    for value in values:
        if value not in possible_hardware:
            possible_hardware.append(value)

{ 'Hardware support': hardware_support_dict, 'Possible values': possible_hardware }

{'Hardware support': {'Llama.cpp': ['NVIDIA GPU',
   'AMD GPU',
   'Apple Silicon',
   'CPU'],
  'ExLlama': ['NVIDIA GPU', 'AMD GPU'],
  'ExLlamav2': ['NVIDIA GPU', 'AMD GPU'],
  'vLLM': ['NVIDIA GPU', 'AMD GPU'],
  'TensorRT-LLM': ['NVIDIA GPU'],
  'TGI': ['AMD GPU', 'NVIDIA GPU', 'Intel GPU', 'AWS Inferentia2'],
  'MLC-LLM': ['NVIDIA GPU',
   'AMD GPU',
   'CPU',
   'WebGPU',
   'Apple Silicon',
   'Intel GPU',
   'WASM',
   'Adreno Mali']},
 'Possible values': ['NVIDIA GPU',
  'AMD GPU',
  'Apple Silicon',
  'CPU',
  'Intel GPU',
  'AWS Inferentia2',
  'WebGPU',
  'WASM',
  'Adreno Mali']}

In [40]:
pip install pandas


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [41]:
import pandas as pd
import numpy as np

df = pd.DataFrame(body, columns=header)

del df['Hardware Support']

for hardware in possible_hardware:
    df[hardware] = df['Method'].apply(lambda method: (
        1 if hardware in hardware_support_dict[method] else 0
    ))

df = df[df['Quantization Type'] != '-']

df.replace('-', np.nan, inplace=True)

df

,Method,Quantization Type,WM (GB),RM (GB),Tokens/sec,Perplexity,NVIDIA GPU,AMD GPU,Apple Silicon,CPU,Intel GPU,AWS Inferentia2,WebGPU,WASM,Adreno Mali
0,Llama.cpp,GGUF K-Quant 2bit,2.36,3.69,102.15,6.96,1,1,1,1,0,0,0,0,0
1,Llama.cpp,GGUF 4bit (check),3.56,4.88,128.97,5.96,1,1,1,1,0,0,0,0,0
2,Llama.cpp,GGUF AWQ 4bit,3.56,4.88,129.25,5.91,1,1,1,1,0,0,0,0,0
3,Llama.cpp,GGUF K-Quant 4bit,3.59,4.90,109.72,5.87,1,1,1,1,0,0,0,0,0
4,Llama.cpp,GGUF 8bit,6.67,7.78,93.39,5.79,1,1,1,1,0,0,0,0,0
5,Llama.cpp,GGUF FP16,12.55,13.22,66.81,5.79,1,1,1,1,0,0,0,0,0
6,ExLlama,GPTQ 4bit,3.63,5.35,77.10,6.08,1,1,0,0,0,0,0,0,0
8,ExLlamav2,EXL2 2bit,2.01,5.21,153.75,20.21,1,1,0,0,0,0,0,0,0
9,ExLlamav2,EXL2 4bit,3.36,6.61,131.68,6.12,1,1,0,0,0,0,0,0,0
10,ExLlamav2,GPTQ 4bit,3.63,6.93,151.30,6.03,1,1,0,0,0,0,0,0,0


In [42]:
df.to_csv('output.csv')